# Prerequisites

1. Load users_001.csv into DataFrame
2. Write the DF as Parquet format
3. Read Parquet file

In [0]:
dbutils.fs.rm("/FileStore/tables/sample_dataset/users_parquet", True)

Out[3]: False

## Create DataFrame

In [0]:
users_df = spark.read.csv(path="/FileStore/tables/sample_dataset/users_csv/", 
               header=True, 
               inferSchema=True)


# Transactio 01 - Write Data as Delta Format

In [0]:
users_df.write.format("delta").mode("overwrite").save("/FileStore/tables/sample_dataset/users_delta")

# Transactio 02 - Overwrite Delta Folder Format

In [0]:
from pyspark.sql.functions import col
users_df.filter(col("city")=="Bangalore").write.format("delta").mode("overwrite").save("/FileStore/tables/sample_dataset/users_delta")

# Transactio 03 - Append Delta Folder Format

In [0]:
from pyspark.sql.functions import col
users_df.filter(col("city")=="Bangalore").write.format("delta").mode("append").save("/FileStore/tables/sample_dataset/users_delta")

# Read the latest version by deault

In [0]:
spark.read.format("delta").load("/FileStore/tables/sample_dataset/users_delta").display()

id,name,dob,email,gender,country,region,city,asset,marital_status
263,Nelle Blake,2024-09-17,nelleblake5744@ymail.com,Transgender,India,Karnataka,Bangalore,908573,Married
263,Nelle Blake,2024-09-17,nelleblake5744@ymail.com,Transgender,India,Karnataka,Bangalore,908573,Married


# Read Transaction Log

## Approach 1

In [0]:
spark.read.format("text").load("dbfs:/FileStore/tables/sample_dataset/users_delta/_delta_log/00000000000000000000.json").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Approach 02

In [0]:
from delta.tables import DeltaTable

delta = DeltaTable.forPath(spark,"dbfs:/FileStore/tables/sample_dataset/users_delta/")
delta.history().display()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-08-09T04:26:59.000+0000,6836536383695527,naveenpn.trainer@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4022668738071548),0809-035608-y0x7rtxm,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 3084)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-08-09T04:20:48.000+0000,6836536383695527,naveenpn.trainer@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4022668738071548),0809-035608-y0x7rtxm,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 3084)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-08-09T04:14:50.000+0000,6836536383695527,naveenpn.trainer@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4022668738071548),0809-035608-y0x7rtxm,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 28541)",null,Databricks-Runtime/12.2.x-scala2.12
